**Connect to Elasticsearch**

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch 

es = Elasticsearch("http://localhost:9200")

client_info = es.info()

print("Connected to Elasticsearch!")

pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**Create index with manual mappings**
#
Because we are using the dense_vector data type, we need to do the mapping manually.

In [3]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties": {
            "side_length": {
                "type": "dense_vector",
                "dims": 4
            },
            "shape": {
                "type": "keyword"
            }
        }
    }
) 

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

**1. Valid case**
#
We specified that sides_length should be a vector with 4 values. Since this condition was met, the operation was executed successfully.

In [4]:
document = {
    "side_length": [5, 5, 5, 5],
    "shape": "square"
}

response = es.index(
    index="my_index", 
    id=1,
    body=document
)

pprint(response.body)

{'_id': '1',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


**2. Invalid case**
#
In this case, the operation fails because `indexing a matrix` is not supported with the `dense_vector` field type.

In [5]:
document = {
    "side_length": [[5, 5], [5, 5]],
    "shape": "square"
}

response = es.index(
    index="my_index",
    id=2,
    body=document
)

pprint(response.body)

BadRequestError: BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [START_ARRAY]', Failed to parse object: expecting token of type [VALUE_NUMBER] but found [START_ARRAY])